In [1]:
import pandas as pd
import numpy as np

import keras as K
import tensorflow as tf
import time

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
class Config(object):
    pass

config = Config()
config.log_dir = 'tf_logs/rnn/'+str(time.strftime('%Y-%m%d %H:%M:%S'))
config.batch_size = 40
config.epochs = 400
config.drop_rate = 0.3

In [3]:
# fix random seed for reproducibility
np.random.seed(42)

In [4]:
from dance.audiosamples import RectifiedAudioSamples as AudioSamples

In [5]:
train_data = pd.read_pickle("train_data.pkl")
val_data = pd.read_pickle("val_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

In [6]:
def generate_samples(train_data, val_data, test_data, sampler):
    try:
        del(train_samples)
        del(val_samples)
        del(test_samples)
    except:
        pass
    train_samples = sampler(train_data, subsample_rate=2)
    scaler = train_samples.norm()
    encoder = train_samples.encode()
    val_samples=sampler(val_data, subsample_rate=2)
    val_samples.norm(scaler)
    val_samples.encode(encoder)
    test_samples = sampler(test_data, subsample_rate=2)
    test_samples.norm(scaler)
    test_samples.encode(encoder)
    return train_samples, val_samples, test_samples

In [7]:
train, val, test = generate_samples(train_data, val_data, test_data, AudioSamples)

In [8]:
tf.reset_default_graph()
model = K.models.Sequential()
model.add(K.layers.SimpleRNN(10, activation='softplus', input_shape=(train.get_as_timeseries().shape[1], train.get_as_timeseries().shape[2]), kernel_regularizer=K.regularizers.l2()))
# model.add(Dropout(config.drop_rate))
# model.add(BatchNormalization())
model.add(K.layers.Dense(100, activation='softplus', kernel_regularizer=K.regularizers.l2()))
model.add(K.layers.Dense(train.Y.shape[-1], activation='softmax', kernel_regularizer=K.regularizers.l2()))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
tbCallback = K.callbacks.TensorBoard(config.log_dir, batch_size=config.batch_size, histogram_freq=10)
patience = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

history = model.fit(train.get_as_timeseries(), train.Y, epochs=config.epochs, 
                    batch_size=config.batch_size, 
                    validation_data=(val.get_as_timeseries(), val.Y), 
                    callbacks=[tbCallback, patience])

Train on 39648 samples, validate on 10166 samples
INFO:tensorflow:Summary name simple_rnn_1/kernel:0 is illegal; using simple_rnn_1/kernel_0 instead.
INFO:tensorflow:Summary name simple_rnn_1/recurrent_kernel:0 is illegal; using simple_rnn_1/recurrent_kernel_0 instead.
INFO:tensorflow:Summary name simple_rnn_1/bias:0 is illegal; using simple_rnn_1/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
Epoch 1/400
39648/39648 [==============================] - 396s - loss: 1.9304 - acc: 0.3002 - val_loss: 1.9400 - val_acc: 0.2589
Epoch 2/400
39648/39648 [==============================] - 359s - loss: 1.7931 - acc: 0.3064 - val_loss: 1.8948 - val_acc: 0.2723
Epoch 3/400
39648/3964

KeyboardInterrupt: 

In [10]:
model.evaluate(test.get_as_timeseries(), y=test.Y, verbose=True)

18384/18384 [==============================] - 67s    


[2.2664258690685473, 0.15573324630113142]